# File arrangement

In [1]:
import os

folder_path = "/work/shared/ngmm/scripts/Beyza_Zayim/Taibur/ngmm-nnunet/dataset/extra_data/file_arrangements"

for file_name in os.listdir(folder_path):
    old_path = os.path.join(folder_path, file_name)
    
    if file_name.endswith("masked.nrrd"):
        parts = file_name.split("_")
        if len(parts) >= 2:
            new_name = f"{parts[0]}_{parts[1]}_0000.nrrd"
            new_path = os.path.join(folder_path, new_name)
            os.rename(old_path, new_path)
            print(f"Renamed: {file_name} -> {new_name}")

    elif file_name.endswith("seg.nrrd"):
        parts = file_name.split("_")
        if len(parts) >= 1:
            new_name = f"{parts[0]}_RCL5.seg.nrrd"
            new_path = os.path.join(folder_path, new_name)
            os.rename(old_path, new_path)
            print(f"Renamed: {file_name} -> {new_name}")


Renamed: NG4995_RCL5_masked.nrrd -> NG4995_RCL5_0000.nrrd
Renamed: NG4995_Segments.seg.nrrd -> NG4995_RCL5.seg.nrrd
Renamed: NG4996_RCL5_masked.nrrd -> NG4996_RCL5_0000.nrrd
Renamed: NG4996_Segments.seg.nrrd -> NG4996_RCL5.seg.nrrd
Renamed: NG4997_RCL5_masked.nrrd -> NG4997_RCL5_0000.nrrd
Renamed: NG4997_Segments.seg.nrrd -> NG4997_RCL5.seg.nrrd


# Path arrangements

In [ ]:
import json
from pathlib import Path
import os
import nibabel as nib
import numpy as np 
import nrrd
import re
from typing import Tuple
import csv
from os.path import join
from utils.helper import convert_file_format, make_if_dont_exist,convert_file_format


In [13]:
# define dataset path
BASE_PATH = Path('./').resolve()
DATA_PATH = BASE_PATH / 'dataset'

project_name = 'first'  # Change here for different task names
task_name = 'Dataset004_' + project_name

TRAINING_DATASET_PATH = BASE_PATH / 'dataset/nnUNet_raw_data' / task_name / 'imagesTr'
GT_TRAINING_DATASET_PATH = BASE_PATH / 'dataset/nnUNet_raw_data' / task_name / 'labelsTr'  # Only nii.gz labels
GT_TRAINING_LABELS_PATH = BASE_PATH / 'dataset/nnUNet_raw_data' / 'extracting_labels'  # Store .nrrd label maps separately
TEST_DATASET_PATH = BASE_PATH / 'dataset/nnUNet_raw_data' / task_name / 'imagesTs'

PREDICTION_RESULTS_PATH = BASE_PATH / 'dataset/nnUNet_Prediction_Results' / task_name
TASK_PATH = BASE_PATH / 'dataset/nnUNet_raw_data' / task_name

# setup environment variables
nnUNet_raw = BASE_PATH / 'dataset/nnUNet_raw_data'
nnUNet_preprocessed = BASE_PATH / 'dataset/nnUNet_preprocessed'
nnUNet_results = BASE_PATH / 'dataset/nnUNet_results'

In [ ]:
make_if_dont_exist(TRAINING_DATASET_PATH,overwrite=False)
make_if_dont_exist(GT_TRAINING_DATASET_PATH)
make_if_dont_exist(TEST_DATASET_PATH)

make_if_dont_exist(PREDICTION_RESULTS_PATH)

make_if_dont_exist(nnUNet_preprocessed)
make_if_dont_exist(nnUNet_results)

In [14]:
train_files = os.listdir(TRAINING_DATASET_PATH)
label_files = os.listdir(GT_TRAINING_DATASET_PATH)
print("train image files:",len(train_files))
print("train label files:",len(label_files))
print("Matches:",len(set(train_files).intersection(set(label_files))))


train image files: 11
train label files: 11
Matches: 0


# Convert File Format to nifti

In [ ]:
convert_file_format(TRAINING_DATASET_PATH, TRAINING_DATASET_PATH, '.nrrd')

In [ ]:

convert_file_format(TEST_DATASET_PATH, TEST_DATASET_PATH, '.nrrd')

In [ ]:

convert_file_format(GT_TRAINING_DATASET_PATH, GT_TRAINING_DATASET_PATH, '.nrrd')

# Check metadata

In [ ]:
import nrrd
import torch

file_path = "/work/shared/ngmm/scripts/Beyza_Zayim/Taibur/ngmm-nnunet/dataset/nrrd/NG2561_Segments_left.seg.nrrd"
data, header = nrrd.read(file_path)

# Convert data to tensor
tensor_data = torch.tensor(data, dtype=torch.float32)
unique_labels = torch.unique(tensor_data)

# Print all metadata
print("NRRD Metadata:")
for key, value in header.items():
    print(f"{key}: {value}")

# Extract label names
label_names = {key.split('_')[0]: value for key, value in header.items() if key.endswith("_Name")}

# Extract corresponding intensity values from LabelValue
intensity_levels = {
    key.split('_')[0]: value for key, value in header.items() if key.endswith("_")
}

# Ensure intensity values are numeric and map them to labels
label_intensity_mapping = {
    label_names[key]: int(intensity_levels[key])
    for key in label_names if key in intensity_levels and str(intensity_levels[key]).isdigit()
}

print(f"{file_path}:", label_intensity_mapping)


# Check labels

In [ ]:


folder_path = GT_TRAINING_LABELS_PATH
output_csv = "/work/shared/ngmm/scripts/Beyza_Zayim/Taibur/ngmm-nnunet/dataset/segment_labels.csv"  # Output CSV file name

# Open the CSV file for writing
with open(output_csv, mode="w", newline="") as csv_file:
    csv_writer = csv.writer(csv_file)
    
    # Write the header row
    csv_writer.writerow(["Filename", "Segment Number", "Label Name"])
    
    # Loop through the folder and process seg.nrrd files
    for file in os.listdir(folder_path):
        if file.endswith("seg.nrrd"):
            file_path = os.path.join(folder_path, file)
            data, header = nrrd.read(file_path)

            # Extract label names
            label_names = {key.split('_')[0]: value for key, value in header.items() if key.endswith("_Name")}
            
            # Process and extract numbers
            for key, value in label_names.items():
                match = re.match(r"Segment(\d+)", key)  # Extract digits after "Segment"
                if match:
                    segment_number = match.group(1)  # Extracted number as string
                    csv_writer.writerow([file, segment_number, value])  # Write to CSV

print(f"CSV file '{output_csv}' has been created successfully!")


CSV file '/work/shared/ngmm/scripts/Beyza_Zayim/Taibur/ngmm-nnunet/dataset/segment_labels.csv' has been created successfully!


In [ ]:


# Function to save JSON
def save_json(data, file_path, sort_keys=False):
    with open(file_path, 'w') as f:
        json.dump(data, f, indent=4, sort_keys=sort_keys)
def generate_dataset_json(output_folder: str,
                          channel_names: dict,
                          labels: dict,
                          num_training_cases: int,
                          file_ending: str,
                          regions_class_order: Tuple[int, ...] = None,
                          dataset_name: str = None, reference: str = None, release: str = None, license: str = None,
                          description: str = None,
                          overwrite_image_reader_writer: str = None, **kwargs):
    
    has_regions: bool = any([isinstance(i, (tuple, list)) and len(i) > 1 for i in labels.values()])
    if has_regions:
        assert regions_class_order is not None, f"You have defined regions but regions_class_order is not set. " \
                                                f"You need that."
    # channel names need strings as keys
    keys = list(channel_names.keys())
    for k in keys:
        if not isinstance(k, str):
            channel_names[str(k)] = channel_names[k]
            del channel_names[k]

    # labels need ints as values
    for l in labels.keys():
        value = labels[l]
        if isinstance(value, (tuple, list)):
            value = tuple([int(i) for i in value])
            labels[l] = value
        else:
            labels[l] = int(labels[l])

    dataset_json = {
        'channel_names': channel_names,  # previously this was called 'modality'. I didn't like this so this is
        # channel_names now. Live with it.
        'labels': labels,
        'numTraining': num_training_cases,
        'file_ending': file_ending,
    }

    if dataset_name is not None:
        dataset_json['name'] = dataset_name
    if reference is not None:
        dataset_json['reference'] = reference
    if release is not None:
        dataset_json['release'] = release
    if license is not None:
        dataset_json['licence'] = license
    if description is not None:
        dataset_json['description'] = description
    if overwrite_image_reader_writer is not None:
        dataset_json['overwrite_image_reader_writer'] = overwrite_image_reader_writer
    if regions_class_order is not None:
        dataset_json['regions_class_order'] = regions_class_order

    dataset_json.update(kwargs)

    save_json(dataset_json, join(output_folder, 'dataset.json'), sort_keys=False)
# Extract label names dynamically from .nrrd files in extracting_labels folder
def extract_labels_from_nrrd(folder_path):
    label_dict = {"background": 0}  # Ensure background is always labeled as 0

    for file in os.listdir(folder_path):
        if file.endswith(".seg.nrrd"):  # Process only segmentation nrrd files
            file_path = os.path.join(folder_path, file)
            data, header = nrrd.read(file_path)

            # Extract segment numbers and names
            for key, value in header.items():
                if key.endswith("_Name"):
                    match = re.match(r"Segment(\d+)", key)  # Extract segment number
                    if match:
                        segment_number = int(match.group(1))
                        if value.lower() != "mask":  # Exclude "mask"
                            label_dict[value] = segment_number  

    return label_dict

# Extract labels dynamically, ensuring "mask" is not included
labels = extract_labels_from_nrrd(GT_TRAINING_LABELS_PATH)
print(labels)

{'background': 0, 'f': 10, 'st': 11, 'ic': 12, 'och': 13, 'ac': 14, 'fr': 15, 'Hb': 16, 'TH': 17, 'MB': 18, 'HY': 19, 'CTX+': 1, 'P': 20, 'MY': 21, 'Cerebellum_AV': 22, 'Cerebellum_GL': 23, 'Cerebellum_ML': 24, 'V': 25, 'OB': 26, 'border': 27, 'cc+': 2, 'CPu': 3, 'DG': 4, 'HP': 5, 'RHP': 6, 'A': 7, 'ig': 8, 'fi': 9}


# Create json file

In [ ]:
# Function to save JSON
def save_json(data, file_path, sort_keys=False):
    with open(file_path, 'w') as f:
        json.dump(data, f, indent=4, sort_keys=sort_keys)
def generate_dataset_json(output_folder: str,
                          channel_names: dict,
                          labels: dict,
                          num_training_cases: int,
                          file_ending: str,
                          regions_class_order: Tuple[int, ...] = None,
                          dataset_name: str = None, reference: str = None, release: str = None, license: str = None,
                          description: str = None,
                          overwrite_image_reader_writer: str = None, **kwargs):
    
    has_regions: bool = any([isinstance(i, (tuple, list)) and len(i) > 1 for i in labels.values()])
    if has_regions:
        assert regions_class_order is not None, f"You have defined regions but regions_class_order is not set. " \
                                                f"You need that."
    # channel names need strings as keys
    keys = list(channel_names.keys())
    for k in keys:
        if not isinstance(k, str):
            channel_names[str(k)] = channel_names[k]
            del channel_names[k]

    # labels need ints as values
    for l in labels.keys():
        value = labels[l]
        if isinstance(value, (tuple, list)):
            value = tuple([int(i) for i in value])
            labels[l] = value
        else:
            labels[l] = int(labels[l])

    dataset_json = {
        'channel_names': channel_names,  # previously this was called 'modality'. I didn't like this so this is
        # channel_names now. Live with it.
        'labels': labels,
        'numTraining': num_training_cases,
        'file_ending': file_ending,
    }

    if dataset_name is not None:
        dataset_json['name'] = dataset_name
    if reference is not None:
        dataset_json['reference'] = reference
    if release is not None:
        dataset_json['release'] = release
    if license is not None:
        dataset_json['licence'] = license
    if description is not None:
        dataset_json['description'] = description
    if overwrite_image_reader_writer is not None:
        dataset_json['overwrite_image_reader_writer'] = overwrite_image_reader_writer
    if regions_class_order is not None:
        dataset_json['regions_class_order'] = regions_class_order

    dataset_json.update(kwargs)

    save_json(dataset_json, join(output_folder, 'dataset.json'), sort_keys=False)
# Extract label names dynamically from .nrrd files in extracting_labels folder
def extract_labels_from_nrrd(folder_path):
    label_dict = {"background": 0}  # Ensure background is always labeled as 0

    for file in os.listdir(folder_path):
        if file.endswith(".seg.nrrd"):  # Process only segmentation nrrd files
            file_path = os.path.join(folder_path, file)
            data, header = nrrd.read(file_path)

            # Extract segment numbers and names
            for key, value in header.items():
                if key.endswith("_Name"):
                    match = re.match(r"Segment(\d+)", key)  # Extract segment number
                    if match:
                        segment_number = int(match.group(1))
                        if value.lower() != "mask":  # Exclude "mask"
                            label_dict[value] = segment_number  

    return label_dict

# Extract labels dynamically, ensuring "mask" is not included
labels = extract_labels_from_nrrd(GT_TRAINING_LABELS_PATH)

# List all nii.gz label files (ignore .nrrd files)
image_files = sorted(os.listdir(TRAINING_DATASET_PATH))
label_files = sorted(f for f in os.listdir(GT_TRAINING_DATASET_PATH) if f.endswith(".nii.gz"))
test_ids = sorted(os.listdir(TEST_DATASET_PATH))

channel_names = {"0": "microscopic"}
num_training_cases = len(image_files)
file_ending = ".nii.gz"

# Generate dataset.json dynamically
generate_dataset_json(
    output_folder=TASK_PATH,
    channel_names=channel_names,
    labels=labels,  # Now "mask" is excluded
    num_training_cases=num_training_cases,
    file_ending=file_ending,
    dataset_name="Mouse Brain Segmentation",
    description="Mouse Brain Segmentation",
    reference="",
    release="0.0",
    license="",
    training=[
        {'image': f"./imagesTr/{image_file}", 'label': f"./labelsTr/{label_file}"}
        for image_file, label_file in zip(image_files, label_files)
    ],
    test=["./imagesTs/%s" % (i[:i.find("_0000")] + '.nii.gz') for i in test_ids]
)


# Remap nifti files (fixing label matches)

In [ ]:
# Function to extract labels from .nrrd
def extract_labels_from_nrrd(folder_path):
    label_dict = {"background": 0}

    for file in os.listdir(folder_path):
        if file.endswith(".seg.nrrd"):
            file_path = os.path.join(folder_path, file)
            data, header = nrrd.read(file_path)

            # Extract segment numbers and names
            for key, value in header.items():
                if key.endswith("_Name"):
                    match = re.match(r"Segment(\d+)", key)
                    if match:
                        segment_number = int(match.group(1))
                        if value.lower() != "mask":  # Exclude "mask"
                            label_dict[value] = segment_number

    return label_dict

# Extract label mapping
labels = extract_labels_from_nrrd(GT_TRAINING_LABELS_PATH)
valid_labels = list(labels.values())  # Get expected label numbers

# Function to re-map incorrect labels in nii.gz files
def remap_labels(nii_path):
    nii = nib.load(nii_path)
    label_data = nii.get_fdata()

    # Check for unexpected labels
    unique_labels = np.unique(label_data)
    unexpected_labels = [l for l in unique_labels if l not in valid_labels and l != 0]

    if unexpected_labels:
        print(f"Fixing labels in: {nii_path}")
        
        # Replace unexpected labels with 0 (or correct mapping)
        fixed_label_data = np.where(np.isin(label_data, unexpected_labels), 0, label_data)
        
        # Save corrected label file
        fixed_nii = nib.Nifti1Image(fixed_label_data, affine=nii.affine, header=nii.header)
        nib.save(fixed_nii, nii_path)
        print(f"Fixed labels in {nii_path}: Removed {unexpected_labels}")

# Apply fix to all `.nii.gz` files
for nii_file in os.listdir(GT_TRAINING_DATASET_PATH):
    if nii_file.endswith(".nii.gz"):
        remap_labels(os.path.join(GT_TRAINING_DATASET_PATH, nii_file))


Fixing labels in: /beegfs/data/work/shared/ngmm/scripts/Beyza_Zayim/Taibur/ngmm-nnunet/dataset/nnUNet_raw_data/Dataset004_first/labelsTr/NG4988_RCL5.nii.gz
Fixed labels in /beegfs/data/work/shared/ngmm/scripts/Beyza_Zayim/Taibur/ngmm-nnunet/dataset/nnUNet_raw_data/Dataset004_first/labelsTr/NG4988_RCL5.nii.gz: Removed [28.0]
Fixing labels in: /beegfs/data/work/shared/ngmm/scripts/Beyza_Zayim/Taibur/ngmm-nnunet/dataset/nnUNet_raw_data/Dataset004_first/labelsTr/NG4990_RCL5.nii.gz
Fixed labels in /beegfs/data/work/shared/ngmm/scripts/Beyza_Zayim/Taibur/ngmm-nnunet/dataset/nnUNet_raw_data/Dataset004_first/labelsTr/NG4990_RCL5.nii.gz: Removed [28.0]
Fixing labels in: /beegfs/data/work/shared/ngmm/scripts/Beyza_Zayim/Taibur/ngmm-nnunet/dataset/nnUNet_raw_data/Dataset004_first/labelsTr/NG4993_RCL5.nii.gz
Fixed labels in /beegfs/data/work/shared/ngmm/scripts/Beyza_Zayim/Taibur/ngmm-nnunet/dataset/nnUNet_raw_data/Dataset004_first/labelsTr/NG4993_RCL5.nii.gz: Removed [28.0]
Fixing labels in: /bee

# Shape match of volume and labels 

In [ ]:
# To check the shape match without saving

# Define paths
image_dir = TRAINING_DATASET_PATH
label_dir = GT_TRAINING_DATASET_PATH


# Get all image and label files
image_files = sorted([f for f in os.listdir(image_dir) if f.endswith(".nii.gz")])
label_files = sorted([f for f in os.listdir(label_dir) if f.endswith(".nii.gz")])

# Extract base names before "_" (e.g., NG2561)
def extract_base_name(filename):
    match = re.match(r"(NG\d+)", filename)  # Match pattern like NG2561
    return match.group(1) if match else None

# Create a mapping of base names to filenames
image_dict = {extract_base_name(f): f for f in image_files}
label_dict = {extract_base_name(f): f for f in label_files}

# Check matching images and segmentations
for base_name in image_dict.keys():
    if base_name in label_dict:
        # Load image and segmentation
        img_path = os.path.join(image_dir, image_dict[base_name])
        seg_path = os.path.join(label_dir, label_dict[base_name])

        img = nib.load(img_path).get_fdata()
        seg = nib.load(seg_path).get_fdata()

        # Print shapes
        print(f"Match Found: {base_name}")
        print("  Image shape:", img.shape)
        print("  Segmentation shape:", seg.shape)
        print("-" * 40)


Match Found: NG2561
  Image shape: (317, 734, 401)
  Segmentation shape: (317, 734, 401)
----------------------------------------
Match Found: NG2562
  Image shape: (299, 784, 409)
  Segmentation shape: (299, 784, 409)
----------------------------------------
Match Found: NG2563
  Image shape: (310, 760, 370)
  Segmentation shape: (310, 760, 370)
----------------------------------------
Match Found: NG2564
  Image shape: (313, 776, 378)
  Segmentation shape: (313, 776, 378)
----------------------------------------
Match Found: NG2565
  Image shape: (313, 797, 382)
  Segmentation shape: (313, 797, 382)
----------------------------------------
Match Found: NG4988
  Image shape: (318, 847, 386)
  Segmentation shape: (319, 847, 387)
----------------------------------------
Match Found: NG4990
  Image shape: (329, 845, 422)
  Segmentation shape: (330, 846, 423)
----------------------------------------
Match Found: NG4993
  Image shape: (325, 865, 410)
  Segmentation shape: (327, 866, 411)
-

In [ ]:
# import nibabel as nib

# img = nib.load('dataset/nnUNet_raw_data/Dataset004_first/imagesTr/NG2561_RCL5_0000.nii.gz')
# seg = nib.load('dataset/nnUNet_raw_data/Dataset004_first/labelsTr/NG2561_RCL5.nii.gz')


# print("Image shape:", img.shape)
# print("Segmentation shape:", seg.shape)

# print("Image voxel spacing:", img.header.get_zooms())
# print("Segmentation voxel spacing:", seg.header.get_zooms())

# print("Image affine:\n", img.affine)
# print("Segmentation affine:\n", seg.affine)


In [ ]:
# To save padded shapes

# Define paths
image_dir = TRAINING_DATASET_PATH
label_dir = GT_TRAINING_DATASET_PATH

# Get all image and label files
image_files = sorted([f for f in os.listdir(image_dir) if f.endswith(".nii.gz")])
label_files = sorted([f for f in os.listdir(label_dir) if f.endswith(".nii.gz")])

# Extract base names before "_" (e.g., NG2561)
def extract_base_name(filename):
    match = re.match(r"(NG\d+)", filename)  # Match pattern like NG2561
    return match.group(1) if match else None

# Create a mapping of base names to filenames
image_dict = {extract_base_name(f): f for f in image_files}
label_dict = {extract_base_name(f): f for f in label_files}

def pad_to_match(image, segmentation):
    """
    Pads the smaller volume to match the shape of the larger volume.
    Assumes only the first dimension (depth) varies.
    """
    img_shape = np.array(image.shape)
    seg_shape = np.array(segmentation.shape)

    if np.array_equal(img_shape, seg_shape):
        return image, segmentation  # Already matching

    pad_width = [(0, 0), (0, 0), (0, 0)]  # Default (no padding)

    for i in range(3):  # Iterate over dimensions
        diff = seg_shape[i] - img_shape[i]
        if diff > 0:  # Image is smaller
            pad_width[i] = (0, diff)
        elif diff < 0:  # Segmentation is smaller
            pad_width[i] = (0, -diff)

    if img_shape[0] < seg_shape[0]:  # Image is smaller in depth
        image = np.pad(image, pad_width, mode='constant', constant_values=0)
    elif img_shape[0] > seg_shape[0]:  # Segmentation is smaller in depth
        segmentation = np.pad(segmentation, pad_width, mode='constant', constant_values=0)

    return image, segmentation

# Loop through matched images and segmentations
for base_name in image_dict.keys():
    if base_name in label_dict:
        img_path = os.path.join(image_dir, image_dict[base_name])
        seg_path = os.path.join(label_dir, label_dict[base_name])

        # Load files
        img_nii = nib.load(img_path)
        seg_nii = nib.load(seg_path)

        img = img_nii.get_fdata()
        seg = seg_nii.get_fdata()

        # Pad to match sizes
        img, seg = pad_to_match(img, seg)

        # Save the fixed files
        nib.save(nib.Nifti1Image(img, img_nii.affine, img_nii.header), img_path)
        nib.save(nib.Nifti1Image(seg, seg_nii.affine, seg_nii.header), seg_path)

        print(f"Fixed & Saved: {base_name}")
        print("  New Image shape:", img.shape)
        print("  New Segmentation shape:", seg.shape)
        print("-" * 40)


Fixed & Saved: NG2561
  New Image shape: (317, 734, 401)
  New Segmentation shape: (317, 734, 401)
----------------------------------------
Fixed & Saved: NG2562
  New Image shape: (299, 784, 409)
  New Segmentation shape: (299, 784, 409)
----------------------------------------
Fixed & Saved: NG2563
  New Image shape: (310, 760, 370)
  New Segmentation shape: (310, 760, 370)
----------------------------------------
Fixed & Saved: NG2564
  New Image shape: (313, 776, 378)
  New Segmentation shape: (313, 776, 378)
----------------------------------------
Fixed & Saved: NG2565
  New Image shape: (313, 797, 382)
  New Segmentation shape: (313, 797, 382)
----------------------------------------
Fixed & Saved: NG4988
  New Image shape: (319, 847, 387)
  New Segmentation shape: (319, 847, 387)
----------------------------------------
Fixed & Saved: NG4990
  New Image shape: (330, 846, 423)
  New Segmentation shape: (330, 846, 423)
----------------------------------------
Fixed & Saved: NG499

In [ ]:
# import nibabel as nib
# import numpy as np

# # Load segmentation
# seg = nib.load('dataset/nnUNet_raw_data/Dataset004_first/labelsTr/NG2561_RCL5.nii.gz')
# seg_data = seg.get_fdata()

# # Pad with 13 slices (assuming missing slices at the top)
# padding = ((13, 0), (0, 0), (0, 0))  # Add 13 slices at the top
# padded_seg_data = np.pad(seg_data, padding, mode='constant', constant_values=0)

# # Save the fixed segmentation
# fixed_seg = nib.Nifti1Image(padded_seg_data, seg.affine, seg.header)
# nib.save(fixed_seg, 'dataset/nnUNet_raw_data/Dataset004_first/labelsTr/NG2561_RCL5.nii.gz')


# Preprocess

In [16]:
%env nnUNet_raw=$nnUNet_raw
%env nnUNet_preprocessed=$nnUNet_preprocessed
%env nnUNet_results=$nnUNet_results

env: nnUNet_raw=/beegfs/data/work/shared/ngmm/scripts/Beyza_Zayim/Taibur/ngmm-nnunet/dataset/nnUNet_raw_data
env: nnUNet_preprocessed=/beegfs/data/work/shared/ngmm/scripts/Beyza_Zayim/Taibur/ngmm-nnunet/dataset/nnUNet_preprocessed
env: nnUNet_results=/beegfs/data/work/shared/ngmm/scripts/Beyza_Zayim/Taibur/ngmm-nnunet/dataset/nnUNet_results


In [26]:
!nnUNetv2_plan_and_preprocess -d 4 --verify_dataset_integrity

Fingerprint extraction...
Dataset004_first
Using <class 'nnunetv2.imageio.simpleitk_reader_writer.SimpleITKIO'> as reader/writer

####################
verify_dataset_integrity Done. 
If you didn't see any error messages then your dataset is most likely OK!
####################

Using <class 'nnunetv2.imageio.simpleitk_reader_writer.SimpleITKIO'> as reader/writer
100%|███████████████████████████████████████████| 11/11 [00:23<00:00,  2.14s/it]
Experiment planning...

############################
INFO: You are using the old nnU-Net default planner. We have updated our recommendations. Please consider using those instead! Read more here: https://github.com/MIC-DKFZ/nnUNet/blob/master/documentation/resenc_presets.md
############################

Attempting to find 3d_lowres config. 
Current spacing: [1.03 1.03 1.03]. 
Current patch size: (112, 224, 80). 
Current median shape: [386.40776699 775.72815534 304.85436893]
Attempting to find 3d_lowres config. 
Current spacing: [1.0609 1.0609 1.060

# Load gcc 9.1

module load gcc/9.1.0